# Running Tune experiments with Optuna

This example demonstrates the usage of Optuna with Ray Tune via `OptunaSearch`, including conditional search spaces and the multi-objective use case.

While you may use a scheduler with `OptunaSearch`, e.g. `AsyncHyperBandScheduler`, please note that schedulers may not work correctly with multi-objective optimization, since they often expect a scalar score.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import time
from typing import Dict, Optional, Any

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch

Toggle `use_func` and `multi_obj` below to test different example usages.

In [ ]:
# Whether to use function to define search space
use_func = False
# Whether to run multi-objective
multi_obj = False

Let's start by defining a simple evaluation function.
The form here is explicit for demonstration, yet it is typically a black-box.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune some hyperparameters,
namely `width`, `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)
              
def multi_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluate(step, config["width"], config["height"], config["activation"])
        # Feed the score back back to Tune.
        tune.report(
           iterations=step, loss=intermediate_score, gain=intermediate_score * width
        )

In [ ]:
ray.init(configure_logging=False)

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparamters may be difficult to find in a short amount of time.

The search space can be defined as independent or conditional. The simplest case is where we have an independent search space. In this case, a config dictionary will suffice.

In [ ]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

However, for the conditional search space, we must use a define-by-run function.

In [ ]:
def define_by_run_func(trial) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``tune.run`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("width", 0, 20)
        trial.suggest_float("height", -100, 100) 
    else:
        trial.suggest_float("width", 0, 21)
        trial.suggest_float("height", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

Optionally, we may provide an initial set of hyperparameters that we believe as the current best, (`current_best_params`), and
pass this belief into a `OptunaSearch` searcher.
Furthermore, we optionally define a `scheduler` to go along with our algorithm.

We then set the maximum concurrent trials to `4` with a `ConcurrencyLimiter`.
Lastly, we must set the number of samples for this Tune run to `1000`
(you can decrease this if it takes too long on your machine).

In [ ]:
initial_params = [
    {"width": 1, "height": 2, "activation": "relu"},
    {"width": 4, "height": 2, "activation": "relu"},
]

# Choosing specified case:
if not multi_obj: 
    if not use_func:
        # Initial guess
        searcher = OptunaSearch(points_to_evaluate=initial_params)
    else:
        # Search space from function, initial guess
        searcher = OptunaSearch(space=define_by_run_func, points_to_evaluate=initial_params)
else: 
    if not use_func:
        # Initial guess, multi-objective
        searcher = OptunaSearch(points_to_evaluate=initial_params, metric=["loss", "gain"], mode=["min", "max"])
    else:
        # Search space from function, initial guess, multi-objective
        searcher = OptunaSearch(space=define_by_run_func, points_to_evaluate=initial_params, metric=["loss", "gain"], mode=["min", "max"])

algo = ConcurrencyLimiter(searcher, max_concurrent=4)
scheduler = AsyncHyperBandScheduler()

num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is running the experiment.

In [ ]:
# Single objective case
if not multi_obj:
    analysis = tune.run(
        objective,
        search_alg=algo,
        scheduler=scheduler,
        metric="mean_loss",
        mode="min",
        num_samples=num_samples,
        config=search_space if not use_func else None
    )

    print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
# Multi-objective case: removed scheduler. metric, mode passed through "search_alg"
if multi_obj:
    analysis = tune.run(
        multi_objective,
        search_alg=algo,
        num_samples=num_samples,
        config=search_space if not use_func else None
    )
    print("Best hyperparameters for loss found were: ", analysis.get_best_config("loss", "min"))
    print("Best hyperparameters for gain found were: ", analysis.get_best_config("gain", "max"))

In [ ]:
ray.shutdown()